# OpenAI Responses API

### Available models:

1. gpt-4 (not used)
2. gpt-4o (currently used)
3. gpt-4o-mini (faster)
4. gpt-4.1 (bigger; slow)
5. gpt-4.1-mini, gpt-4.1-nano

### Available reasoning models:

1. o4-mini (best reasoning; slow)
2. o3 (medium reasoning)
3. o1 (medium reasoning)
4. o3-mini (least reasoning; fast)

In [ ]:
from openai import OpenAI
client = OpenAI(api_key="")

In [17]:
# OpenAI Response API

response = client.responses.create(
    model="gpt-4o",
    input="Write a one-sentence bedtime story about a unicorn."
)

print(response.output_text)

Once there was a unicorn named Luna who painted the night sky with her shimmering horn, filling everyone's dreams with colors never seen before.


In [ ]:
# Instructions mode

response = client.responses.create(
    model="gpt-4o-mini",
    instructions="Talk like a pirate.",
    input="Are semicolons optional in JavaScript?",
)

print(response.output_text)

Arrr, matey! When it comes to JavaScript, semicolons be mostly optional thanks to somethin’ called Automatic Semicolon Insertion (ASI). The JavaScript engine be tryin’ to save yer hide by puttin’ them in where it thinks they belong. But beware, ye scallywag! Sometimes, skipparoo semicolons can cause tricky bugs that be hard to spot on the high seas of code.

So, while ye *can* sail without semicolons in many cases, it be wise to drop ‘em at the end of statements to keep yer ship steady and yer code clear. Ye best beware the places ASI might steer ye wrong, like return statements or lines startin’ with parentheses or square brackets!

In short, semicolons be optional but recommended, lest ye want yer code to walk the plank! Arrr!


In [ ]:
# Using Reasoning Models

response = client.responses.create(
    model="",
    reasoning={"effort": "low"}, # possible modes: low, medium, high
    instructions="Talk like a pirate.",
    input="Are semicolons optional in JavaScript?",
)

print(response.output_text)

In [18]:
# Explicit instructions mode where role can be specified

response = client.responses.create(
    model="gpt-4.1",
    # reasoning={"effort": "low"},
    input=[
        {
            "role": "developer",
            "content": "Talk like a pirate."
        },
        {
            "role": "user",
            "content": "Are semicolons optional in JavaScript?"
        }
    ]
)

print(response.output_text)

Arrr, ye be wonderin’ about them pesky semicolons, eh? In the land o’ JavaScript, semicolons *can* be optional, thanks to a sneaky crew known as Automatic Semicolon Insertion (or **ASI** fer short). JavaScript’ll often be smart enough to patch things up if ye forget a semicolon at the end o’ yer line. Fer example:

```javascript
let booty = "gold"
console.log(booty)
```

That’ll work just fine, matey! The interpreter’ll slip in a semicolon when ye miss one.

**But beware!** Sometimes the wind don’t fill yer sails and strange bugs appear if ye always leave those semicolons behind! Some lines, like these here:

```javascript
return
{
    booty: "gold"
}
```

Will return *undefined*, not yer precious gold, because the interpreter throws in a semicolon *after* `return`! Arrr, beware that trap, lest yer code end up in Davy Jones’ locker!

**Rule o’ thumb:** Most pirates (er... programmers) drop semicolons when writin’ simple things, but use ‘em religious-like to keep surprises at bay. So, *

In [ ]:
# Structured outputs

from pydantic import BaseModel

"""
Supported types for Structured outputs:

String
Number
Boolean
Integer
Object
Array
Enum
anyOf
"""

class CalendarEvent(BaseModel): # this is the structured output class
    name: str
    date: str
    participants: list[str]

response = client.responses.parse(
    model="o1",
    input=[
        {"role": "system", "content": "Extract the event information."},
        {
            "role": "user",
            "content": "Alice and Bob are going to a science fair on Friday.",
        },
    ],
    text_format=CalendarEvent,
)

event = response.output_parsed
print(event)

name='Science fair' date='Friday' participants=['Alice', 'Bob']


In [ ]:
# WebSearch Tool

response = client.responses.create(
    model="gpt-4o",
    tools=[{"type": "web_search"}],
    input="What was a positive news story from today?"
)

print(response.output_text)

Here’s a heartening and uplifting news story from **today, September 16, 2025**:

An endangered British wetland creature, the **water vole**, has been reintroduced into the wild—marking a hopeful milestone in conservation efforts. The species, which had been absent from many of its native habitats, is now showing **promising signs of recovery** thanks to dedicated rewilding programs. This development represents much-needed progress in restoring ecosystem balance and biodiversity.([thecooldown.com](https://www.thecooldown.com/outdoors/water-vole-reintroduction-uk-ecosystem-restoration/?utm_source=openai))

Additionally, today marks **World Ozone Day**, a globally recognized observance that highlights the success of international efforts to protect the ozone layer. This year’s commemoration is particularly significant—it celebrates **40 years of the Vienna Convention**, an international treaty adopted in 1985 to safeguard Earth’s ozone layer. The day serves as a powerful reminder of the 

In [ ]:
# Streaming responses

with client.responses.stream(
    model="gpt-4.1",
    input=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a short poem about the sea."},
    ],
) as stream:
    for event in stream:
        if event.type == "response.output_text.delta":
            print(event.delta, end="")   # stream the text chunks
        elif event.type == "response.error":
            print("Error:", event.error)
        elif event.type == "response.completed":
            print("\nCompleted")

    # If you want the final aggregated response:
    final_response = stream.get_final_response()
    print("\n\nFinal Response:", final_response.output_text)

Endless blue, a rolling song,  
Where timeless waves sweep dreams along.  
Salt-kissed air and seagulls’ cries,  
Mirrors of the shifting skies.

Secrets deep and stories old,  
Whispered where the tides unfold.  
The sea, alive, both wild and free—  
Calling hearts to mystery.
Completed


Final Response: Endless blue, a rolling song,  
Where timeless waves sweep dreams along.  
Salt-kissed air and seagulls’ cries,  
Mirrors of the shifting skies.

Secrets deep and stories old,  
Whispered where the tides unfold.  
The sea, alive, both wild and free—  
Calling hearts to mystery.
